In [1]:
import pandas as pd
import re
from time import time

import sys
reload(sys)
sys.setdefaultencoding('utf8')

In [2]:
!pwd

In [3]:
# !ls /home/saya/notebook/twitter/

In [4]:
df=pd.read_csv('/home/saya/notebook/twitter/twitter_basement.csv')
df.shape

(452275, 7)

In [5]:
test=df['text'][0]

In [6]:
print test

In [7]:
def clean(test):
    return re.sub(r'@\S+|#|RT|\n|http\S+','',str(test)).strip()

In [8]:
a = df['text'][29]
a

'Sundays Open House. #Realtor #Pool #Basement #Acre #RuralArea https://t.co/ELNiaVcvQs'

In [9]:
content = df['text'].apply(clean)

In [10]:
len(content)

452275

### topic modeling

In [11]:
import nltk
import string
from nltk.stem.wordnet import WordNetLemmatizer

# nltk.download('stopwords')
from nltk.corpus import stopwords 
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()


In [12]:
stop = pd.read_fwf('stop.txt',names=['words'])
stopwords.update(set(stop['words'].values))
# print (list(stopwords)[:10])
stopwords.update(set(['dont','lot','wa','im','ha','doe','wa', 'able',]))

In [13]:
def cleaning(article):
    article = str(article)
    zero = "".join(i for i in article if i not in punctuation)
    
    one = " ".join([i for i in zero.lower().split() if i not in stopwords])
    
    three = " ".join(lemmatize.lemmatize(i) for i in one.split())
    return three

In [14]:
all_content=content.copy()
apple = all_content

In [15]:
# nltk.download('wordnet')
start = time()

ap_text = apple.apply(cleaning)
ap_text_list = [i.split() for i in ap_text]
print (len(ap_text_list))

('used: {:.2f}s'.format(time()-start))

'used: 50.57s'

In [16]:
ap_text[:10]

0    videogamesday dreamed converting basement arca...
1           2player challenge eventide island basement
2                         misogynist live mom basement
3    kink lost abandoned house amp hearing tune pia...
4    2k spade domino mode join godsc jewel inc toni...
5                         misogynist live mom basement
6        yo basement story pinegroves dope found base���
7    mini child programming basement sweatshop produce
8    videogamesday dreamed converting basement arca...
9                            mind laundry basement def
Name: text, dtype: object

In [17]:
from gensim.models.phrases import Phraser, Phrases

start = time()
phs = Phrases(ap_text_list)
('used: {:.2f}s'.format(time()-start))

'used: 5.69s'

In [18]:
bi_gram = Phraser(phs)
start = time()

new_bi_list = [bi_gram[i] for i in ap_text_list]

('used: {:.2f}s'.format(time()-start))

'used: 11.04s'

In [19]:
start = time()

phs3=Phrases(new_bi_list)
tri_gram=Phraser(phs3)

('used: {:.2f}s'.format(time()-start))

'used: 7.51s'

In [20]:
start = time()

new_tri_list = [tri_gram[i] for i in new_bi_list]

('used: {:.2f}s'.format(time()-start))

'used: 7.76s'

In [21]:
len(new_tri_list)

452275

In [22]:
new_tri_list[0]

[u'videogamesday_dreamed_converting',
 u'basement',
 u'arcade_youre_retrogames\u2026']

In [23]:
make_dic = dict()
for i,j in zip(range(len(new_tri_list)), new_tri_list):
    make_dic[i]=j

In [24]:
without_stop = pd.Series(make_dic)
without_stop.head()

0    [videogamesday_dreamed_converting, basement, a...
1     [2player, challenge, eventide, island, basement]
2                      [misogynist_live_mom, basement]
3    [kink_lost_abandoned_house, amp_hearing_tune_p...
4    [2k_spade_domino_mode, join_godsc_jewel_inc, t...
dtype: object

In [25]:
ap_text_list = list(without_stop.values)

In [26]:
len(ap_text_list)

452275

In [27]:
ap_text_list[100:103]

[[u'broadcaster',
  u'criticism',
  u'stagebut',
  u'guy',
  u'eating',
  u'soup',
  u'mom',
  u'basement',
  u'look',
  u'mirror\u2026'],
 [u'nice', u'basement'],
 [u'buddy', u'basement', u'fuck']]

### start building

In [28]:
# Importing Gensim
import gensim
from gensim import corpora

from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='twitter_running.log',filemode='w')

In [29]:
# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(ap_text_list)
dictionary.save('twitter.dict')
(dictionary)

In [30]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in ap_text_list]
corpora.MmCorpus.serialize('twitter.mm', doc_term_matrix)

(len(doc_term_matrix))
(doc_term_matrix[100])

[(2, 1),
 (32, 1),
 (66, 1),
 (331, 1),
 (332, 1),
 (333, 1),
 (334, 1),
 (335, 1),
 (336, 1),
 (337, 1)]

In [40]:
start = time()
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=30, id2word = dictionary, 
#                passes=5
              )
'used: {:.2f}s'.format(time()-start)

'used: 631.15s'

In [32]:
ldamodel.print_topics()

[(0,
  u'0.138*"basement" + 0.037*"gonna" + 0.017*"house" + 0.014*"home" + 0.012*"look" + 0.011*"live" + 0.011*"amp" + 0.010*"locked" + 0.009*"mom" + 0.008*"mommy"'),
 (1,
  u'0.033*"survivalist_filled_massive" + 0.033*"food_\u2014_decided_puerto" + 0.032*"ricans" + 0.019*"post_malone_look" + 0.019*"sitting_smoke_circle" + 0.019*"70_\U0001f602\U0001f602" + 0.013*"embracing_blessing_god" + 0.013*"happ\u2026" + 0.011*"amp_letting_write_height" + 0.011*"thank_providing_artistic_home"'),
 (2,
  u'0.087*"embracing_blessing_god" + 0.023*"america_trump_supporter_\U0001f1fa\U0001f1f8" + 0.022*"church_boycott_nfl_right\u2026" + 0.015*"it\u2019s" + 0.015*"crazed_lunatic_left_condoning" + 0.015*"shooting_look_loser_live\u2026" + 0.015*"violence_amp_appears_mass" + 0.014*"someone" + 0.012*"wo_cry" + 0.012*"repeat_dark"'),
 (3,
  u'0.153*"basement" + 0.013*"door" + 0.011*"3" + 0.010*"finished" + 0.009*"gun" + 0.009*"please" + 0.009*"real" + 0.008*"buildi\u2026" + 0.008*"there\u2019s_talk_including_

In [33]:
ldamodel.save('twitter.model')

### load saved model , plotting

In [34]:
from gensim.models import LdaModel
loading = LdaModel.load('twitter.model')

In [35]:
import pyLDAvis.gensim
import gensim
from time import time
pyLDAvis.enable_notebook()

In [36]:
d = gensim.corpora.Dictionary.load('twitter.dict')
c = gensim.corpora.MmCorpus('twitter.mm')
lda = gensim.models.LdaModel.load('twitter.model')

In [37]:
start = time()

data = pyLDAvis.gensim.prepare(lda, c, d)

('used: {:.2f}s'.format(time()-start))

'used: 186.52s'

In [38]:
pyLDAvis.save_html(data,'twitter.html')


In [39]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      12.215191        1       1  0.068454 -0.101729
0      12.155633        1       2  0.132480  0.068365
4      11.787159        1       3  0.135455  0.033417
9      11.030407        1       4  0.100853  0.025065
8      10.719702        1       5 -0.208301  0.375103
7       9.689509        1       6  0.099656  0.036826
5       9.495119        1       7  0.137631 -0.052660
2       9.028272        1       8 -0.312824 -0.144251
6       8.111696        1       9  0.151489 -0.100303
1       5.767312        1      10 -0.304894 -0.139832, topic_info=      Category           Freq                                  Term  \
term                                                                  
86365  Default  223993.000000                              basement   
9700   Default   21535.000000                                period   
6289   Default   15786.000000                embracing_blessing_god   
47861  Default   13090.000000                ima_steal_money_church   
17200  Default   12093.000000   lord_watchingbecause_that���s_exactly   
24753  Default    8100.000000                                 gonna   
7744   Default    5677.000000               ���_can���t_believe_they���re   
80957  Default    5677.000000              wild_cat_spent_someone���s   
40084  Default    5633.000000             actually_feeling_grass_a���   
25183  Default    5001.000000                                  2011   
65886  Default    7143.000000                                   mom   
89365  Default    6776.000000                 food_���_decided_puerto   
12170  Default    6808.000000            survivalist_filled_massive   
27985  Default    6671.000000                                ricans   
37274  Default    4092.000000                           house_music   
36566  Default    6356.000000                                   amp   
25593  Default    3722.000000            america_trump_supporter_��������   
42830  Default    3712.000000             church_boycott_nfl_right���   
24637  Default    3443.000000                           spread_love   
12986  Default    3443.000000         hello_creative_project_called   
81270  Default    3670.000000            logic_talking_updating_fan   
42233  Default    3833.000000                                 house   
29102  Default    3471.000000                                   kid   
11894  Default    3274.000000                                people   
52066  Default    3147.000000                                 party   
51762  Default    3230.000000                               he���s_27   
62800  Default    3230.000000                       mcm_tweeted_mom   
11786  Default    3036.000000                                  life   
33799  Default    3001.000000                                  home   
48014  Default    2868.000000                                  door   
...        ...            ...                                   ...   
74993  Topic10     458.040383                       story_neck_deep   
69207  Topic10     458.040383      knocked_loose_turnstile_movement   
16560  Topic10     464.909988  lamelo_ball_schedule510am_basketball   
56340  Topic10     464.909988  winter_merch_basement13pmonline_math   
28976  Topic10     464.909988               101pm_knit_baller_brand   
93508  Topic10     454.618997                                  sick   
41562  Topic10     429.349859                               hillary   
21330  Topic10     414.706079                                racist   
59647  Topic10     393.129868                               keeping   
8184   Topic10     361.997469                                  joke   
92383  Topic10     332.216967                                thread   
28673  Topic10     310.493097                                 adult   
76733  Topic10     297.612011                                  dick   
39643  Topic10     